#### Testų žymėjimas (@pytest.mark)

In [ ]:
#Pagrindai apie @pytest.mark
#    1. Žymėjimas testų funkcijų ar klasių:
import pytest

@pytest.mark.slow
def test_slow_function():
    # lėtas testas
    pass

@pytest.mark.smoke
def test_quick_smoke():
    # greitas smoke testas
    pass

@pytest.mark.skip(reason="Dar neįgyvendintas")
def test_skip_this():
    pass
#    2. Testų filtravimas pagal žymes vykdymo metu:
# Pytest leidžia vykdyti tik tam tikrus žymėtus testus:
pytest -m slow
# arba vykdyti visus testus išskyrus tam tikrus:
pytest -m "not slow"


In [ ]:
# Dažnai naudojamos žymos:
# - @pytest.mark.skip(reason="priežastis") – praleidžia testą su pateikta priežastimi.
# - @pytest.mark.skipif(condition, reason="priežastis") – praleidžia testą, jei sąlyga tenkinama.
# - @pytest.mark.parametrize – leidžia parametrizuoti testus su skirtingais duomenimis (truputį kitokia paskirtis).
# - Custom žymos – gali susikurti savas, pvz. @pytest.mark.api, @pytest.mark.db.

In [ ]:
# Kaip susikurti savo žymes (custom markers)
# Pytest rekomenduoja savo žymes užregistruoti pytest.ini faile, kad neliktų įspėjimų:
# pytest.ini:
[pytest]
markers =
    slow: Lėti testai
    smoke: Greiti pagrindiniai testai
    api: Testai, kurie tikrina API

# Pavyzdys su custom žymomis:
import pytest

@pytest.mark.slow
def test_very_slow():
    assert True

@pytest.mark.smoke
def test_quick():
    assert True

@pytest.mark.api
def test_api_call():
    assert True


#### @pytest.mark.slow 

In [ ]:
# @pytest.mark.slow yra pytest markeris, kuris naudojamas pažymėti lėtus testus – tokius,
# kurie užtrunka ilgiau nei įprasti. Tai leidžia greitai atskirti ir valdyti testų vykdymą 
# pagal greitį.
import pytest
import time

@pytest.mark.slow
def test_long_process():
    time.sleep(5)  # imituojame lėtą veiksmą
    assert True

# Kombinuotas pavyzdys:
import pytest

@pytest.mark.slow
def test_labas():
    assert 1 + 1 == 2

@pytest.mark.smoke
def test_greitas():
    assert 2 * 2 == 4
    
# Kada naudoti @pytest.mark.slow
# Kai testas:
# - Naudoja išorinį API.
# - Atlieka didelį skaičiavimą.
# - Dirba su duomenų baze ar failų sistema.
# - Užtrunka ilgiau nei kelias sekundes.

#### @pytest.mark.skipif(condition, reason="priežastis")

In [ ]:
#@pytest.mark.skipif(condition, reason="priežastis") – tai pytest dekoratorius, leidžiantis
# praleisti testą, jeigu nurodyta sąlyga yra True. Tai naudinga, kai testas priklauso nuo tam
# tikrų aplinkybių – pvz., platformos, Python versijos, bibliotekos buvimo ar pan.

# Sintaksė
import pytest

@pytest.mark.skipif(condition, reason="kodėl testas praleidžiamas")
def test_something():
    # testas bus praleistas, jei condition == True
    ...

# Pavyzdžiai:
# Praleisti testą, jei naudojamas Windows:
import pytest
import sys

@pytest.mark.skipif(sys.platform.startswith("win"), reason="Nedirba Windows sistemoje")
def test_linux_only_feature():
    assert True
    
# Praleisti testą, jei Python versija < 3.9:
import pytest
import sys

@pytest.mark.skipif(sys.version_info < (3, 9), reason="Reikalingas Python 3.9 arba naujesnis")
def test_new_syntax():
    assert True

# Praleisti, jei nėra tam tikros bibliotekos:
import pytest

try:
    import numpy as np
    has_numpy = True
except ImportError:
    has_numpy = False

@pytest.mark.skipif(not has_numpy, reason="Reikalinga numpy biblioteka")
def test_numpy_functionality():
    assert True


#### @pytest.mark.xfail

In [ ]:
# @pytest.mark.xfail – tai pytest dekoratorius, kuris žymi testą kaip 
# „tikėtinai nepavyksiantį“ („expected to fail“). Kitaip tariant, tu žinai,
# kad testas šiuo metu neveiks, bet nenori, kad jis būtų laikomas tikru klaidos atveju.

# Pagrindinė sintaksė
import pytest

@pytest.mark.xfail
def test_known_bug():
    assert 1 == 2  # Testas nepraeis, bet pytest to nelaikys klaida

# Kada naudoti xfail
# - Kai žinai apie esamą klaidą (bug), bet dar jos neištaisėte.
# - Kai funkcionalumas dar neįgyvendintas, bet testas jau parašytas.
# - Kai norite įtraukti testą į CI/CD, bet žinoti, kad jis kol kas gali nepraeiti.

# Naudingos parinktys
# 1. Su reason
@pytest.mark.xfail(reason="Yra žinoma klaida #123")
def test_broken():
    assert 1 == 2
    
# 2. Su condition (pvz., tik tam tikrose sistemose)
import sys

@pytest.mark.xfail(sys.platform == "win32", reason="Nepalaikomas Windows")
def test_only_unix():
    assert True

# 3. Su strict=True – kad XPASS būtų laikomas klaida
@pytest.mark.xfail(strict=True)
def test_should_fail():
    assert 1 == 1  # Testas "praėjo" – bus laikoma KLAIDA

# Naudojimas su @pytest.mark.parametrize
import pytest

@pytest.mark.parametrize("x", [1, 2, 3])
def test_values(x):
    assert x < 3

# Pritaikytas vienam atvejui:
@pytest.mark.parametrize("x", [
    1,
    pytest.param(3, marks=pytest.mark.xfail(reason="Per didelis")),
])
def test_xfail_one_case(x):
    assert x < 3


#### Autouse fixture’ai (be aiškaus kvietimo)

In [ ]:
# autouse fixture’us – tai specialaus tipo pytest fixture’ai, kurie paleidžiami 
# automatiškai, net jei nėra aiškiai paminėti teste.
# autouse=True nurodo pytest, kad šis fixture būtų automatiškai naudojamas visuose
# atitinkamo lygio testuose, be to, kad jį reiktų perduoti kaip argumentą į testą.

# Paprastas pavyzdys:
import pytest

@pytest.fixture(autouse=True)
def setup_teardown():
    print("🔧 Setup")
    yield
    print("🧹 Teardown")

def test_example_1():
    print("🧪 Test 1")
    assert True

def test_example_2():
    print("🧪 Test 2")
    assert True
# Output bus:
🔧 Setup
🧪 Test 1
🧹 Teardown
🔧 Setup
🧪 Test 2
🧹 Teardown

# Kada naudoti autouse=True?
# | Naudojimo situacija                               | Ar tinka `autouse=True`?    |
# | ------------------------------------------------- | --------------------------- |
# | Testų aplinkos paruošimas (failai, katalogai, DB) | ✅                           |
# | Laikino katalogo sukūrimas, valymas               | ✅                           |
# | Aiškių duomenų tiekimas testui                    | ❌ (naudok paprastą fixture) |

# Geriausia praktika:
# - Naudok autouse tik tada, kai visi testai failo/module/klasės lygyje tikrai to fixture reikalaus.
# - Nenaudok, jei fixture skirtas tik kai kuriems testams – geriau perduok jį kaip argumentą.

# Scope kombinavimas
# Gali derinti su scope:
@pytest.fixture(autouse=True, scope="module")
def connect_to_db():
    print("📡 Prisijungta prie DB")
    yield
    print("❌ Atsijungta nuo DB")


#### Testų išimtis (pytest.raises) 

In [ ]:
# pytest.raises – tai pytest funkcija, skirta patikrinti, ar kodas iškelia tam tikrą išimtį (exception). 
# Ji labai naudinga testuojant klaidų tvarkymą ar validaciją, kai tikiesi, kad tam tikromis sąlygomis kodas 
# turi sugeneruoti klaidą (pvz., ValueError, TypeError ir pan.).

# Pagrindinė sintaksė
import pytest

def test_divide_by_zero():
    with pytest.raises(ZeroDivisionError):
        1 / 0
# Testas bus sėkmingas, jei iš tikrųjų bus pakelta ZeroDivisionError. Jei ne – testas nepavyks.

# Patikrinti pranešimo (message) tekstą
def raise_error():
    raise ValueError("Netinkamas formatas")

def test_error_message():
    with pytest.raises(ValueError, match="Netinkamas formatas"):
        raise_error()

# Naudojimas su kontekstu (as exc_info)
# Galima naudoti as norint toliau tikrinti klaidos objektą:
def test_exception_details():
    with pytest.raises(ValueError) as exc_info:
        int("abc")  # klaida konvertuojant tekstą į skaičių

    assert "invalid literal" in str(exc_info.value)

# Parametrizuotas pavyzdys
import pytest

@pytest.mark.parametrize("input", ["", "abc", "1.5"])
def test_int_conversion_raises(input):
    with pytest.raises(ValueError):
        int(input)


In [ ]:
# Užduotis: Funkcija „get_element“
# Sukurk funkciją get_element, kuri:
# Priima sąrašą (list) ir indeksą (int)
# Grąžina reikšmę pagal nurodytą indeksą
# Jei indeksas neteisingas (per mažas arba per didelis), turi iškelti IndexError
#
# Reikalavimai:
# Parašyk funkciją get_element(list_, index)
# Parašyk du testus:
# Vienas testas, kai indeksas teisingas → turi grąžinti reikšmę
# Kitas testas, kai indeksas neteisingas → turi iškelti IndexError (naudok pytest.raises)

# confest.py
import pytest

def get_element(list_, index):
    return  list_[index]

import pytest

# test_my_function.py
from tests.conftest import get_element
def test_good_index():
    assert get_element(["a", "b", "c"], 1) == "b"

def test_bad_element():
    with pytest.raises(IndexError):
        get_element(["a", "b", "c"], 5)

In [ ]:
# užduotis:
# Sukurkime funkciją, kuri padalija du skaičius, bet jei antras skaičius yra 0,
# turi būti iškelta ZeroDivisionError.

# conftest.py
import pytest

def divine(a, b):
    if b == 0:
        raise ZeroDivisionError("Dalyba negalima")
    return a/b

# test_+my_function.py
import pytest

from tests.conftest import divine


def test_divine_good():
    assert divine(6, 3) == 2

def test_divine_bad():
    with pytest.raises(ZeroDivisionError):
        divine(5,0)